In [1]:
from utils.spark_session import get_spark_session
from pyspark.sql import SparkSession

spark: SparkSession = get_spark_session()

25/10/18 08:26:35 WARN Utils: Your hostname, DESKTOP-9VM3SA1 resolves to a loopback address: 127.0.1.1; using 172.28.82.250 instead (on interface eth0)
25/10/18 08:26:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/dottier/venvs/flights_env/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/dottier/.ivy2/cache
The jars for the packages stored in: /home/dottier/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bfa1287c-cecf-4eb1-83bb-9eb95138103b;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.postgresql#postgresql;42.7.3 in central
	found org.checkerframework#checker-qual;3.42.0 in central
:: resolution report :: resolve -1689ms :: artifacts dl 9ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.2.0 from central in [default]
	io.delta#delta-storage;3.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	org.checkerframework#checker-qual;3.42.0 from central in [default]
	org.postgresql#postgresql;42.7.3 from central in [default]
	---------------------------

In [4]:
spark.sql("SHOW TABLES in bronze").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|   bronze| airports|      false|
|   bronze|  flights|      false|
+---------+---------+-----------+



In [3]:
from delta.tables import DeltaTable

bronze_flights = DeltaTable.forName(spark, "bronze.flights")
bronze_flights.history().show(5)

25/10/12 17:28:28 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      1|2025-10-12 17:22:...|  NULL|    NULL|    WRITE|{mode -> Overwrit...|NULL|    NULL|     NULL|          0|  Serializable|        false|{numFiles -> 16, ...|        NULL|Apache-Spark/3.5....|
|      0|2025-10-12 16:13:...|  NULL|    NULL|    WRITE|{mode -> Overwrit...|NULL|    NULL|     NULL|       NULL|  Serializable|        false|{numFiles -> 158,...|        NULL|Apache-Spark/3.5....|
+-------+-

In [4]:
# Roll back the table to version 14
bronze_flights.restoreToVersion(0)


25/10/12 17:29:17 WARN DAGScheduler: Broadcasting large task binary with size 1076.9 KiB
25/10/12 17:29:19 WARN DAGScheduler: Broadcasting large task binary with size 1122.5 KiB
25/10/12 17:29:23 WARN DAGScheduler: Broadcasting large task binary with size 1159.9 KiB


DataFrame[table_size_after_restore: bigint, num_of_files_after_restore: bigint, num_removed_files: bigint, num_restored_files: bigint, removed_files_size: bigint, restored_files_size: bigint]

In [6]:
spark.sql("SHOW TABLES in silver").show()

+---------+-------------+-----------+
|namespace|    tableName|isTemporary|
+---------+-------------+-----------+
|   silver| dim_aircraft|      false|
|   silver|dim_aircrafts|      false|
|   silver| dim_airlines|      false|
|   silver| dim_airports|      false|
|   silver|  dim_runways|      false|
|   silver|  fct_flights|      false|
+---------+-------------+-----------+



In [7]:
spark.sql("DROP DATABASE silver CASCADE")

25/10/14 06:48:01 WARN TxnHandler: Cannot perform cleanup since metastore table does not exist


DataFrame[]

In [2]:
from utils.io_utils import read_df

silver_flights = read_df(spark, "silver", "fct_flights")
print(silver_flights.count())

silver_runways = read_df(spark, "silver", "dim_runways")
silver_runways.count()

25/10/14 17:21:17 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/10/14 17:21:17 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
25/10/14 17:21:19 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
25/10/14 17:21:19 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore dottier@127.0.1.1
25/10/14 17:21:19 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
25/10/14 17:21:21 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


183756


163

In [6]:
silver_runways.count()

163

In [ ]:
silver_runways.show(200)


+--------------------+--------------------+-----------+------------+--------+------------+---------+-----------+----------+------------------------+---------+-----------+--------------------+------------------+
|           runway_sk|          airport_sk|runway_name|true_heading| surface|has_lighting|is_closed|length_feet|width_feet|displaced_threshold_feet| latitude|  longitude|effective_start_date|effective_end_date|
+--------------------+--------------------+-----------+------------+--------+------------+---------+-----------+----------+------------------------+---------+-----------+--------------------+------------------+
| 5018015408876227825| 4450828643872079020|         12|       117.7| Asphalt|        true|    false|    10285.0|     148.0|                     0.0|36.691074|  -4.507827|2025-10-14 06:49:...|              NULL|
| 1597118613431100831| 4450828643872079020|         13|       131.7| Asphalt|        true|    false|    10515.0|     148.0|                     0.0| 36.6845

In [5]:
import pyspark.sql.functions as sf

silver_runways.select(
    sf.count_distinct(sf.col("airport_sk"))
).show()

+--------------------------+
|count(DISTINCT airport_sk)|
+--------------------------+
|                        31|
+--------------------------+



In [4]:
a = silver_flights.dropDuplicates(["flight_composite_pk"])
a.count()

183756

In [9]:
silver_airports = read_df(spark, "silver", "dim_airports")

In [10]:
silver_airports.printSchema()

root
 |-- airport_sk: long (nullable = true)
 |-- airport_iata: string (nullable = true)
 |-- airport_icao: string (nullable = true)
 |-- airport_name: string (nullable = true)
 |-- municipality_name: string (nullable = true)
 |-- country_name: string (nullable = true)
 |-- continent_name: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- elevation_feet: double (nullable = true)
 |-- airport_time_zone: string (nullable = true)



In [ ]:
from load.database import transaction_context

with transaction_context() as db:
    db.execute("""
        CREATE TABLE dim_airports (
            airport_sk BIGINT PRIMARY KEY,
            airport_iata VARCHAR(10),
            airport_icao VARCHAR(10),
            airport_name VARCHAR(255),
            municipality_name VARCHAR(255),
            country_name VARCHAR(100),
            continent_name VARCHAR(50),
            latitude DOUBLE PRECISION,
            longitude DOUBLE PRECISION,
            elevation_feet DOUBLE PRECISION,
            airport_time_zone VARCHAR(100)
        );
    """)

ERROR:root:Error in database: relation "dim_airports" already exists
Traceback (most recent call last):
  File "/home/dottier/flights_etl/load/database.py", line 26, in get_db
    yield cur
  File "/tmp/ipykernel_1758/4159816736.py", line 4, in <module>
    db.execute("""
psycopg2.errors.DuplicateTable: relation "dim_airports" already exists



In [ ]:
with transaction_context() as db:
    db.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public';")
    result = db.fetchone()
    print(result)

('dim_airports',)


In [3]:
from utils.io_utils import read_df
from load.utils import load_table_to_postgres

silver_airports = read_df(spark, "silver", "dim_airports")
silver_runways = read_df(spark, "silver", "dim_runways")
silver_airlines = read_df(spark, "silver", "dim_airlines")
silver_aircrafts = read_df(spark, "silver", "dim_aircrafts")
silver_flights = read_df(spark, "silver", "fct_flights")


25/10/18 08:26:58 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/10/18 08:26:58 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
25/10/18 08:26:59 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
25/10/18 08:26:59 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore dottier@127.0.1.1
25/10/18 08:27:00 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
25/10/18 08:27:01 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
dim_airports_sql = f"""
    CREATE TABLE dim_airports (
        airport_sk BIGINT PRIMARY KEY,
        airport_iata VARCHAR(10),
        airport_icao VARCHAR(10),
        airport_name VARCHAR(255),
        municipality_name VARCHAR(255),
        country_name VARCHAR(100),
        continent_name VARCHAR(50),
        latitude DOUBLE PRECISION,
        longitude DOUBLE PRECISION,
        elevation_feet DOUBLE PRECISION,
        airport_time_zone VARCHAR(100)
    );
"""

dim_airlines_sql = f"""
    CREATE TABLE dim_airlines (
        airline_sk BIGINT PRIMARY KEY,
        airline_iata VARCHAR(10),
        airline_icao VARCHAR(10),
        airline_name VARCHAR(255)
    );
"""

dim_aircrafts_sql = f"""
    CREATE TABLE dim_aircrafts (
        aircraft_sk BIGINT PRIMARY KEY,
        aircraft_reg VARCHAR(100),
        aircraft_mode_s VARCHAR(100),
        aircraft_model VARCHAR(100),
        aircraft_sk_type VARCHAR(100)
    );
"""

dim_runways_sql = f"""
    CREATE TABLE dim_runways (
        runway_sk BIGINT PRIMARY KEY,
        airport_sk BIGINT REFERENCES dim_airports (airport_sk),
        runway_name VARCHAR(100),
        true_heading DOUBLE PRECISION,
        surface VARCHAR(100),
        has_lighting BOOLEAN,
        is_closed BOOLEAN,
        length_feet DOUBLE PRECISION,
        width_feet DOUBLE PRECISION,
        displaced_threshold_feet DOUBLE PRECISION,
        latitude DOUBLE PRECISION,
        longitude DOUBLE PRECISION,
        effective_start_Date TIMESTAMP,
        effective_end_date TIMESTAMP
    );
"""

dim_flight_details_sql = f"""
    CREATE TABLE dim_flight_details (
        flight_details_sk SERIAL PRIMARY KEY,
        flight_status TEXT NOT NULL,
        codeshare_status TEXT NOT NULL,
        is_cargo BOOLEAN NOT NULL,
        
        -- This ensures we only store each unique combination once
        UNIQUE (flight_status, codeshare_status, is_cargo)
    );
"""

dim_quality_sql = f"""
    CREATE TABLE dim_quality_combination (
        quality_combo_sk SERIAL PRIMARY KEY,
        dep_has_basic BOOLEAN NOT NULL,
        dep_has_live BOOLEAN NOT NULL,
        arr_has_basic BOOLEAN NOT NULL,
        arr_has_live BOOLEAN NOT NULL,

        quality_desc TEXT,

        -- Unique constraint on the flags
        UNIQUE (dep_has_basic, dep_has_live, arr_has_basic, arr_has_live)
    );
"""


fct_flights_sql = f"""
    CREATE TABLE fct_flights (
        -- Surrogate Key for performance
        fact_flight_id BIGSERIAL PRIMARY KEY,
        
        -- Natural Key for business logic & upserts
        flight_composite_pk TEXT NOT NULL,

        -- === FOREIGN KEYS ===
        flight_details_sk INT REFERENCES dim_flight_details(flight_details_sk), -- The new FK
        departure_airport_sk BIGINT REFERENCES dim_airports(airport_sk),
        arrival_airport_sk BIGINT REFERENCES dim_airports(airport_sk),
        airline_sk BIGINT REFERENCES dim_airlines(airline_sk),
        aircraft_sk BIGINT REFERENCES dim_aircrafts(aircraft_sk),
        departure_runway_sk BIGINT REFERENCES dim_runways(runway_sk),
        arrival_runway_sk BIGINT REFERENCES dim_runways(runway_sk),
        quality_combo_sk INT REFERENCES dim_quality_combination(quality_combo_sk),

        -- === DEGENERATE DIMENSIONS (Attributes that stay in the fact table) ===
        flight_number TEXT,      -- High cardinality, belongs here
        flight_callsign TEXT,    -- High cardinality, belongs here
        dep_terminal TEXT,
        arr_gate TEXT,
        dep_checkin_desk TEXT,
        dep_gate TEXT,
        arr_terminal TEXT,
        arr_baggage_belt TEXT,
        dep_local_timezone TEXT,
        arr_local_timezone TEXT,

        -- === MEASURES / FACTS ===
        dep_scheduled_at_utc TIMESTAMP,
        dep_revised_at_utc TIMESTAMP,
        dep_runway_at_utc TIMESTAMP,
        dep_scheduled_at_local_ntz TIMESTAMP,
        dep_revised_at_local_ntz TIMESTAMP,
        dep_runway_at_local_ntz TIMESTAMP,
        arr_scheduled_at_utc TIMESTAMP,
        arr_revised_at_utc TIMESTAMP,
        arr_runway_at_utc TIMESTAMP,
        arr_scheduled_at_local_ntz TIMESTAMP,
        arr_revised_at_local_ntz TIMESTAMP,
        arr_runway_at_local_ntz TIMESTAMP,
        
        last_location_reported_at_utc TIMESTAMP,
        latitude DOUBLE PRECISION,
        longitude DOUBLE PRECISION,
        altitude_ft DOUBLE PRECISION,
        ground_speed_kts DOUBLE PRECISION,
        true_track_deg DOUBLE PRECISION,
        ingestion_hour TEXT,

        -- A unique constraint on the natural key to enforce business uniqueness
        UNIQUE (flight_composite_pk)
    );
"""

In [7]:
from load.utils import transaction_context

with transaction_context() as cur:
    # Create tables with schema
    cur.execute(dim_airports_sql)
    cur.execute(dim_airlines_sql)
    cur.execute(dim_aircrafts_sql)
    cur.execute(dim_runways_sql)
    cur.execute(dim_flight_details_sql)
    cur.execute(dim_quality_sql)
    cur.execute(fct_flights_sql)

ERROR:root:Error in database: relation "dim_airports" already exists
Traceback (most recent call last):
  File "/home/dottier/flights_etl/load/database.py", line 26, in get_db
    yield cur
  File "/home/dottier/flights_etl/load/database.py", line 49, in transaction_context
    yield new_cursor
  File "/tmp/ipykernel_67767/1577498256.py", line 5, in <module>
    cur.execute(dim_airports_sql)
psycopg2.errors.DuplicateTable: relation "dim_airports" already exists



In [ ]:
        airport_sk BIGINT PRIMARY KEY,
        airport_iata VARCHAR(10),
        airport_icao VARCHAR(10),
        airport_name VARCHAR(255),
        municipality_name VARCHAR(255),
        country_name VARCHAR(100),
        continent_name VARCHAR(50),
        latitude DOUBLE PRECISION,
        longitude DOUBLE PRECISION,
        elevation_feet DOUBLE PRECISION,
        airport_time_zone VARCHAR(100)

In [2]:
from load.utils import load_to_postgres, read_df_from_postgres
from load.database import transaction_context

with transaction_context() as master_cursor:
    load_to_postgres(
        spark=spark, df=silver_airports, target_table="dim_airports",
        strategy="upsert", pk_cols=["airport_sk"], cursor=master_cursor
    )
    insert_unknown_airport_sql = f"""
        INSERT INTO dim_airports (airport_sk, airport_iata, airport_icao, airport_name, municipality_name, country_name, continent_name, latitude, longitude, elevation_feet, airport_time_zone)
        VALUES (-1, 'UNK', 'UNK', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 0, 0, 0, 'Unknown')
        ON CONFLICT (airport_sk) DO NOTHING;
    """
    master_cursor.execute(insert_unknown_airport_sql)

    load_to_postgres(
        spark=spark, df=silver_airlines, target_table="dim_airlines",
        strategy="upsert", pk_cols=["airline_sk"], cursor=master_cursor
    )
    load_to_postgres(
        spark=spark, df=silver_aircrafts, target_table="dim_aircrafts",
        strategy="upsert", pk_cols=["aircraft_sk"], cursor=master_cursor
    )

    # unknown_runway = [
    #     {"airport_sk": -1, "runway_name": "UNKNOWN", "true_heading": 0.0, "surface": "Unknown",
    #     "has_lighting": False, "is_closed": False, "length_feet": 0.0, "width_feet": 0.0, "displaced_threshold_feet": 0.0,
    #     "latitude": 0.0, "longitude": 0.0}
    # ]

    load_to_postgres(
        spark=spark, df=silver_runways, target_table="dim_runways",
        strategy="upsert", pk_cols=["runway_sk"], cursor=master_cursor
    )
    insert_unknown_runway_sql = f"""
        INSERT INTO dim_runways (
            runway_sk, airport_sk, runway_name, true_heading, surface, has_lighting,
            is_closed, length_feet, width_feet, displaced_threshold_feet,
            latitude, longitude, effective_start_date, effective_end_date
        )
        VALUES (
            -1, -1, 'UNK', 0.0, 'Unknown', False,
            False, 0.0, 0.0, 0.0,
            0.0, 0.0, '1970-01-01', '9999-12-31'
        )
        ON CONFLICT (runway_sk) DO NOTHING;
    """
    master_cursor.execute(insert_unknown_runway_sql)

ERROR:root:Error in database: name 'silver_airports' is not defined
Traceback (most recent call last):
  File "/home/dottier/flights_etl/load/database.py", line 26, in get_db
    yield cur
  File "/home/dottier/flights_etl/load/database.py", line 49, in transaction_context
    yield new_cursor
  File "/tmp/ipykernel_1058/2310015354.py", line 6, in <module>
    spark=spark, df=silver_airports, target_table="dim_airports",
                    ^^^^^^^^^^^^^^^
NameError: name 'silver_airports' is not defined


In [4]:
from load.database import transaction_context

with transaction_context() as cur:
    cur.execute(
        "TRUNCATE TABLE dim_airports, dim_airlines, dim_aircrafts, dim_runways, dim_flight_details, dim_quality_combination, fct_flights;"
    )

In [35]:
silver_runways.filter("airport_sk = -1").show(50, truncate=False)

+-------------------+----------+-----------+------------+-------+------------+---------+-----------+----------+------------------------+--------+---------+--------------------------+------------------+
|runway_sk          |airport_sk|runway_name|true_heading|surface|has_lighting|is_closed|length_feet|width_feet|displaced_threshold_feet|latitude|longitude|effective_start_date      |effective_end_date|
+-------------------+----------+-----------+------------+-------+------------+---------+-----------+----------+------------------------+--------+---------+--------------------------+------------------+
|8077286841863878235|-1        |UNKNOWN    |0.0         |Unknown|false       |false    |0.0        |0.0       |0.0                     |0.0     |0.0      |2025-10-13 16:49:22.618962|NULL              |
+-------------------+----------+-----------+------------+-------+------------+---------+-----------+----------+------------------------+--------+---------+--------------------------+----------

In [15]:
silver_flights.printSchema()

root
 |-- flight_composite_pk: string (nullable = true)
 |-- departure_airport_sk: long (nullable = true)
 |-- arrival_airport_sk: long (nullable = true)
 |-- airline_sk: long (nullable = true)
 |-- aircraft_sk: long (nullable = true)
 |-- departure_runway_sk: long (nullable = true)
 |-- arrival_runway_sk: long (nullable = true)
 |-- flight_number: string (nullable = true)
 |-- flight_callsign: string (nullable = true)
 |-- flight_status: string (nullable = true)
 |-- codeshare_status: string (nullable = true)
 |-- is_cargo: boolean (nullable = true)
 |-- dep_scheduled_at_utc: timestamp (nullable = true)
 |-- dep_revised_at_utc: timestamp (nullable = true)
 |-- dep_runway_at_utc: timestamp (nullable = true)
 |-- dep_scheduled_at_local_ntz: timestamp (nullable = true)
 |-- dep_revised_at_local_ntz: timestamp (nullable = true)
 |-- dep_runway_at_local_ntz: timestamp (nullable = true)
 |-- dep_local_timezone: string (nullable = true)
 |-- arr_scheduled_at_utc: timestamp (nullable = true)


In [19]:
from pyspark.sql.functions import array_contains, concat, lit, col, array_join, format_string

#dim_flight_details_df = silver_flights.

# dim_flight_details_sql = f"""
#     CREATE TABLE dim_flight_details (
#         flight_details_sk SERIAL PRIMARY KEY,
#         flight_status TEXT NOT NULL,
#         codeshare_status TEXT NOT NULL,
#         is_cargo BOOLEAN NOT NULL,
        
#         -- This ensures we only store each unique combination once
#         UNIQUE (flight_status, codeshare_status, is_cargo)
#     );
# """

dim_flight_details_df = silver_flights.select(
    "flight_status", "codeshare_status", "is_cargo"
).distinct()

dim_quality_combination_df = silver_flights.select(
    array_contains("dep_quality", "Basic").alias("dep_has_basic"),
    array_contains("dep_quality", "Live").alias("dep_has_live"),
    array_contains("arr_quality", "Basic").alias("arr_has_basic"),
    array_contains("arr_quality", "Live").alias("arr_has_live"),
).distinct()

dim_quality_combination_df = dim_quality_combination_df.withColumn(
    "quality_desc",
    format_string(
        "Dep B:%s L:%s, Arr B:%s L:%s",
        col("dep_has_basic"), col("dep_has_live"),
        col("arr_has_basic"), col("arr_has_live")
    )
)

dim_quality_combination_df.show(10, truncate=False)


+-------------+------------+-------------+------------+---------------------------------------+
|dep_has_basic|dep_has_live|arr_has_basic|arr_has_live|quality_desc                           |
+-------------+------------+-------------+------------+---------------------------------------+
|true         |false       |true         |true        |Dep B:true L:false, Arr B:true L:true  |
|true         |true        |false        |false       |Dep B:true L:true, Arr B:false L:false |
|true         |true        |true         |true        |Dep B:true L:true, Arr B:true L:true   |
|true         |false       |false        |false       |Dep B:true L:false, Arr B:false L:false|
|false        |false       |true         |true        |Dep B:false L:false, Arr B:true L:true |
|false        |false       |true         |false       |Dep B:false L:false, Arr B:true L:false|
|true         |false       |true         |false       |Dep B:true L:false, Arr B:true L:false |
|true         |true        |true        

In [ ]:
fct_flights_df = (
    silver_flights
    .join(
        dim_flight_details_df, on=["flight_status", "codeshare_status", "is_cargo"], how="left"
    )
    .join(
        dim_quality_combination_df, on=["dep_quality", "arr_quality"], how="left"
    )
)

In [11]:
fct_flights_df.show(10, truncate=False)

+-------------+-------------+-------------+----------------+--------+---------------------------------------------------------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-----------------+-------------+---------------+--------------------+-------------------+-----------------+--------------------------+------------------------+-----------------------+------------------+--------------------+-------------------+-------------------+--------------------------+------------------------+-----------------------+------------------+------------+----------------+--------+----------+------------+--------+----------------+----------+-----------------------------+--------+---------+-----------+----------------+--------------+--------------+-------------+------------+-------------+------------+-----------------------------+
|dep_quality  |arr_quality  |flight_status|codeshare_status|is_cargo|flight_composite_pk                  

In [ ]:
with transaction_context() as cur:
    cur.execute("""
        ALTER TABLE dim_flight_details
        ADD 
    """)

ERROR:root:Error in database: syntax error at end of input
LINE 3:     
            ^
Traceback (most recent call last):
  File "/home/dottier/flights_etl/load/database.py", line 26, in get_db
    yield cur
  File "/home/dottier/flights_etl/load/database.py", line 49, in transaction_context
    yield new_cursor
  File "/tmp/ipykernel_1058/2660416548.py", line 2, in <module>
    cur.execute("""
psycopg2.errors.SyntaxError: syntax error at end of input
LINE 3:     
            ^



In [17]:
from load.utils import load_to_postgres, read_df_from_postgres
from load.database import transaction_context

with transaction_context() as master_cursor:
    load_to_postgres(
        spark=spark, df=dim_flight_details_df, target_table="dim_flight_details",
        strategy="upsert", pk_cols=["flight_status", "codeshare_status", "is_cargo"], cursor=master_cursor
    )

    load_to_postgres(
        spark=spark, df=dim_quality_combination_df, target_table="dim_quality_combination",
        strategy="upsert", pk_cols=["dep_has_basic", "dep_has_live", "arr_has_basic", "arr_has_live"], cursor=master_cursor
    )

In [ ]:
quality_lookup_df = read_df_from_postgres(spark, "dim_quality_combination")
flight_details_lookup_df = read_df_from_postgres(spark, "dim_flight_details")

quality_lookup_df.show(5)
flight_details_lookup_df.show(5)

+----------------+-------------+------------+-------------+------------+--------------------+
|quality_combo_sk|dep_has_basic|dep_has_live|arr_has_basic|arr_has_live|        quality_desc|
+----------------+-------------+------------+-------------+------------+--------------------+
|               1|         true|       false|         true|        true|Dep B:true L:fals...|
|               2|         true|        true|        false|       false|Dep B:true L:true...|
|               3|         true|        true|         true|        true|Dep B:true L:true...|
|               4|         true|       false|        false|       false|Dep B:true L:fals...|
|               5|        false|       false|         true|        true|Dep B:false L:fal...|
+----------------+-------------+------------+-------------+------------+--------------------+
only showing top 5 rows

+-----------------+-------------+----------------+--------+
|flight_details_sk|flight_status|codeshare_status|is_cargo|
+--------

In [22]:
fct_flights_df = silver_flights.withColumn(
    "dep_has_basic", array_contains(col("dep_quality"), "Basic")
).withColumn(
    "dep_has_live", array_contains(col("dep_quality"), "Live")
).withColumn(
    "arr_has_basic", array_contains(col("arr_quality"), "Basic")
).withColumn(
    "arr_has_live", array_contains(col("arr_quality"), "Live")
)

fct_flights_df = (
    fct_flights_df
    .join(
        flight_details_lookup_df,
        on=["flight_status", "codeshare_status", "is_cargo"],
        how="left"
    )
    .join(
        quality_lookup_df,
        on=["dep_has_basic", "dep_has_live", "arr_has_basic", "arr_has_live"],
        how="left"
    )
)

In [23]:
fct_flights_df.show(10)

+-------------+------------+-------------+------------+-------------+----------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-----------------+-------------+---------------+--------------------+-------------------+-----------------+--------------------------+------------------------+-----------------------+------------------+--------------------+-------------------+-------------------+--------------------------+------------------------+-----------------------+------------------+------------+----------------+--------+----------+-------------+------------+--------+----------------+----------+-------------+-----------------------------+--------+---------+-----------+----------------+--------------+--------------+-----------------+----------------+--------------------+
|dep_has_basic|dep_has_live|arr_has_basic|arr_has_live|flight_status|codeshare_status|is_cargo| flight_composite_pk|departure_air

In [ ]:
with transaction_context() as master_cursor:
    load_to_postgres(
        spark=spark, df=fct_flights_df, target_table="fct_flights",
        strategy="delete_insert", cursor=master_cursor
    )

In [14]:
import importlib
import load.utils  # make sure the module is imported
importlib.reload(load.utils)



ImportError: cannot import name 'transaction_context' from 'load.database' (/home/dottier/flights_etl/load/database.py)

In [3]:
silver_flights.printSchema()

root
 |-- flight_composite_pk: string (nullable = true)
 |-- departure_airport_sk: long (nullable = true)
 |-- arrival_airport_sk: long (nullable = true)
 |-- airline_sk: long (nullable = true)
 |-- aircraft_sk: long (nullable = true)
 |-- departure_runway_sk: long (nullable = true)
 |-- arrival_runway_sk: long (nullable = true)
 |-- flight_number: string (nullable = true)
 |-- flight_callsign: string (nullable = true)
 |-- flight_status: string (nullable = true)
 |-- codeshare_status: string (nullable = true)
 |-- is_cargo: boolean (nullable = true)
 |-- dep_scheduled_at_utc: timestamp (nullable = true)
 |-- dep_revised_at_utc: timestamp (nullable = true)
 |-- dep_runway_at_utc: timestamp (nullable = true)
 |-- dep_scheduled_at_local_ntz: timestamp (nullable = true)
 |-- dep_revised_at_local_ntz: timestamp (nullable = true)
 |-- dep_runway_at_local_ntz: timestamp (nullable = true)
 |-- dep_local_timezone: string (nullable = true)
 |-- arr_scheduled_at_utc: timestamp (nullable = true)


In [6]:
import pyspark.sql.functions as sf
silver_flights.groupBy(sf.col("arr_quality")).agg(
    sf.count(sf.col("arr_quality")).alias("count")
).show()

+-------------+------+
|  arr_quality| count|
+-------------+------+
|           []|  7147|
|[Basic, Live]|133071|
|      [Basic]| 43538|
+-------------+------+



In [3]:
silver_flights.show(10, truncate=False)

+---------------------------------------------------------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-----------------+-------------+---------------+-------------+----------------+--------+--------------------+-------------------+-----------------+--------------------------+------------------------+-----------------------+------------------+--------------------+-------------------+-------------------+--------------------------+------------------------+-----------------------+------------------+------------+----------------+--------+----------+-------------+------------+--------+----------------+----------+-------------+-----------------------------+--------+---------+-----------+----------------+--------------+--------------+
|flight_composite_pk                                                  |departure_airport_sk|arrival_airport_sk  |airline_sk          |aircraft_sk         |departure_runway_sk|arrival_runway_

In [4]:
import pyspark.sql.functions as sf

silver_flights.groupBy(sf.col("flight_number")).agg(
    sf.count(sf.col("flight_number")).alias("count")
).orderBy(sf.col("count").desc()).show()

+-------------+-----+
|flight_number|count|
+-------------+-----+
|        HBJWC|   23|
|       UA 195|   18|
|       EK 206|   18|
|       ET 737|   18|
|       EK 255|   18|
|       EK 210|   18|
|       EK 209|   18|
|       UA 160|   18|
|        UA 45|   18|
|        UA 59|   18|
|        DL 21|   18|
|       AY 804|   18|
|       EK 256|   18|
|       ET 736|   18|
|        UA 41|   18|
|       AY 803|   18|
|        SQ 26|   18|
|        UA 31|   18|
|        SQ 25|   18|
|        AA 71|   17|
+-------------+-----+
only showing top 20 rows



In [14]:
silver_flights.filter("flight_number = 'EK 206'").show(10, truncate=False)

+-------------------------------------------------------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+-------------+---------------+-------------+----------------+--------+--------------------+-------------------+-------------------+--------------------------+------------------------+-----------------------+------------------+--------------------+-------------------+-----------------+--------------------------+------------------------+-----------------------+------------------+------------+----------------+--------+----------+-------------+------------+--------+----------------+----------+-------------+-----------------------------+--------+---------+-----------+----------------+--------------+--------------+
|flight_composite_pk                                                |departure_airport_sk|arrival_airport_sk  |airline_sk          |aircraft_sk         |departure_runway_sk|arrival_runway_sk

In [12]:
silver_runways.filter("runway_sk = 8077286841863878235").show(20, truncate=False)

+-------------------+----------+-----------+------------+-------+------------+---------+-----------+----------+------------------------+--------+---------+--------------------------+------------------+
|runway_sk          |airport_sk|runway_name|true_heading|surface|has_lighting|is_closed|length_feet|width_feet|displaced_threshold_feet|latitude|longitude|effective_start_date      |effective_end_date|
+-------------------+----------+-----------+------------+-------+------------+---------+-----------+----------+------------------------+--------+---------+--------------------------+------------------+
|8077286841863878235|-1        |UNKNOWN    |0.0         |Unknown|false       |false    |0.0        |0.0       |0.0                     |0.0     |0.0      |2025-10-13 16:49:22.618962|NULL              |
+-------------------+----------+-----------+------------+-------+------------+---------+-----------+----------+------------------------+--------+---------+--------------------------+----------

In [11]:
silver_airports.filter("airport_sk IN (-3869769487129258055, -7941055854170734276, 7883337853819725554 )").show(20, truncate=False)

+--------------------+------------+------------+-----------------+-----------------+------------+--------------+--------+---------+--------------+-----------------+
|airport_sk          |airport_iata|airport_icao|airport_name     |municipality_name|country_name|continent_name|latitude|longitude|elevation_feet|airport_time_zone|
+--------------------+------------+------------+-----------------+-----------------+------------+--------------+--------+---------+--------------+-----------------+
|-7941055854170734276|ZRH         |LSZH        |Zurich Kloten    |Zurich           |Switzerland |Europe        |47.4647 |8.549169 |1416.0        |Europe/Zurich    |
|-3869769487129258055|PMI         |LEPA        |Palma De Mallorca|Palma De Mallorca|Spain       |Europe        |39.5517 |2.73881  |27.0          |Europe/Madrid    |
|7883337853819725554 |BSL         |LFSB        |Bâle/Mulhouse    |NULL             |NULL        |NULL          |NULL    |NULL     |NULL          |Europe/Paris     |
+---------

In [8]:
silver_airlines.printSchema()
silver_aircrafts.printSchema()
silver_runways.printSchema()

root
 |-- airline_sk: long (nullable = true)
 |-- airline_iata: string (nullable = true)
 |-- airline_icao: string (nullable = true)
 |-- airline_name: string (nullable = true)

root
 |-- aircraft_sk: long (nullable = true)
 |-- aircraft_reg: string (nullable = true)
 |-- aircraft_mode_s: string (nullable = true)
 |-- aircraft_model: string (nullable = true)
 |-- aircraft_sk_type: string (nullable = true)

root
 |-- runway_sk: long (nullable = true)
 |-- airport_sk: long (nullable = true)
 |-- runway_name: string (nullable = true)
 |-- true_heading: double (nullable = true)
 |-- surface: string (nullable = true)
 |-- has_lighting: boolean (nullable = true)
 |-- is_closed: boolean (nullable = true)
 |-- length_feet: double (nullable = true)
 |-- width_feet: double (nullable = true)
 |-- displaced_threshold_feet: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- effective_start_date: timestamp (nullable = true)
 |-- effective_e

In [ ]:
with transaction_context() as cur:
    db.execute("""
        CREATE TABLE dim_runways (
            runway_sk BIGINT PRIMARY KEY,
            airport_sk BIGINT REFERENCES dim_airports (airport_sk),
            runway_name VARCHAR(100),
            true_heading DOUBLE PRECISION,
            surface VARCHAR(100),
            has_lighting BOOLEAN,
            is_closed BOOLEAN,
            length_feet DOUBLE PRECISION
            width_feet DOUBLE PRECISION,
            displaced_threshold_feet DOUBLE PRECISION,
            latitude DOUBLE PRECISION,
            longitude DOUBLE PRECISION,
            effective_start_Date TIMESTAMP,
            effective_end_date TIMESTAMP
        );
    """)

In [ ]:
with transaction_context() as db:
    # Print schema for a specific table, e.g., dim_airports
    table_name = 'dim_airports'
    db.execute("""
        SELECT column_name, data_type, is_nullable
        FROM information_schema.columns
        WHERE table_schema = 'public'
          AND table_name = %s;
    """, (table_name,))
    for col in db.fetchall():
        print(col)

In [3]:
silver_airports.filter("airport_iata = 'LHR'").show()

+--------------------+------------+------------+---------------+-----------------+--------------+--------------+--------+---------+--------------+-----------------+
|          airport_sk|airport_iata|airport_icao|   airport_name|municipality_name|  country_name|continent_name|latitude|longitude|elevation_feet|airport_time_zone|
+--------------------+------------+------------+---------------+-----------------+--------------+--------------+--------+---------+--------------+-----------------+
|-7580325845725858894|         LHR|        EGLL|London Heathrow|           London|United Kingdom|        Europe| 51.4706|-0.461941|          83.0|    Europe/London|
+--------------------+------------+------------+---------------+-----------------+--------------+--------------+--------+---------+--------------+-----------------+



In [4]:
from load.utils import load_to_postgres

load_to_postgres(spark, silver_airports, "dim_airports", "upsert", pk_cols=["airport_sk"])

In [6]:
bronze_airports_df = read_df(spark, "bronze", "airports")
bronze_airports_df.show(40)

+----+----+---------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+------------+-----------------+--------------------+--------------------+--------------------+-------+
|icao|iata|localCode|           shortName|            fullName| municipalityName|            location|           elevation|             country|   continent|         timeZone|                urls|             runways|         currentTime|airport|
+----+----+---------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+------------+-----------------+--------------------+--------------------+--------------------+-------+
|LEMG| AGP|     NULL|              Málaga|              Málaga|           Málaga| {36.6749, -4.49911}|{16.15, 0.02, 0.0...|         {ES, Spain}|{EU, Europe}|    Europe/Madrid|{http://www.aena....|[{12, 117.7, {313...|{{2025-10-04 15:2...|    AGP|
|EHAM| AMS| 